# Mergesort

**Mergesort** is one of two classic sorting problems (along with quicksort) that is still heavily used even 50 years later. They are critical components in the world's computational infrastructure.

The basic idea for mergesort is to divide the array into two halves, recursively sort each half, then merge the two halves. Merging is the key here, given two sorted subarrays `a[lo]` to `a[mid]` and `a[mid+1]` to `a[hi]`, your goal is to replace them with a sorted subarray `a[lo]` to `a[hi]`.

Good practice to put assertions into your code to check that what you expect is true (here that the merge function is expecting the two subarrays are actually sorted). You can enable or disable at runtime - by default they're disabled.

Next step is to implement the **sort** part of the procedure, which recursively splits the array in half, then merges the pieces in order.

In [1]:
def merge(arr, mid):
    '''
    Assumes there's a midpoint mid such that the subarray from the indices
    0 to mid is sorted and the subarray from indices mid+1 to the end is
    sorted. Returns the sorted array
    '''
    if len(arr) == 1:
        return arr
    lo = 0
    hi = len(arr) - 1
#     assert all(arr[i] <= arr[i+1] for i in range(lo, mid))
#     assert all(arr[j] <= arr[j+1] for j in range(mid + 1, hi))
    
    aux = arr[lo:hi + 1]
    i, j = lo, mid + 1
    for k in range(lo, hi + 1):
        if i > mid:
            # i index is exhausted
            arr[k] = aux[j]
            j += 1
        elif j > hi:
            # j index is exhausted
            arr[k] = aux[i]
            i += 1
        elif aux[j] < aux[i]:
            arr[k] = aux[j]
            j += 1
        else:
            arr[k] = aux[i]
            i += 1

    return arr

a = ['A', 'G', 'L', 'O', 'R', 'H', 'I', 'M', 'S', 'T']
mid = 4

print("Merge: {}".format(merge(a, mid)))

# Force an assertion error
# b = ['G', 'A', 'L', 'R', 'O', 'H', 'I', 'M', 'S', 'T']
# merge(b, mid)

Merge: ['A', 'G', 'H', 'I', 'L', 'M', 'O', 'R', 'S', 'T']


In [2]:
def mergeSort(arr):
    if len(arr) == 1:
        return arr
    else:
        mid = (len(arr) - 1) // 2
        return merge(mergeSort(arr[:mid + 1]) + mergeSort(arr[mid + 1:]), mid)


c = ['A', 'G', 'L', 'O', 'R', 'H', 'I', 'M', 'S', 'T']
mergeSort(c)

['A', 'G', 'H', 'I', 'L', 'M', 'O', 'R', 'S', 'T']

## Mergesort Analysis

Mergesort is a **divide and conquer** algorithm. Good algorithms are more effective than super computers - the empirical analysis shows insertion sort would take 37 years on an average home computer for 1 billion items, where mergesort would take a week or so.

The proposition is that mergesort uses at most $N \lg N$ compares and $6 N \lg N$ array accesses to sort any array of size $N$.

**Running time**: You can draw a recurrence relation - the number of compares $C(N)$ and array accesses $A(N)$ to mergesort an array of size $N$ satisfy the recurrences:

$$
C(N) \leq C([N/2]) + C[N/2]) + N \text{ for } N \gt 1 \text{, with } C(1) = 0 \\
A(N) \leq A([N/2]) + A[N/2]) + 6N \text{ for } N \gt 1 \text{, with } A(1) = 0
$$

**Memory usage**: Mergesort uses extra space proportional to $N$ because the `aux[]` array needs to be of size $N$ for the last merge. **NOT in place**.

**Practical improvements**:

- Use insertion sort for small subarrays, mergesort has too much overhead for tiny subarrays. Have a cutoff to use insertion sort (7 items or less)
- Stop if already sorted - this happens if the biggest item in the first half is $\leq$ the smallest item in the second half. This helps for partially sorted arrays
- Eliminate the copy to the auxiliary array (saves time, not space) by switching the role of the input and auxiliary arrays in each recursive call

## Bottom-up Mergesort

**Botton-up mergesort** is a version without recursion - it starts with mini subarrays of size one, merges them into order by twos, then fours, etc.

There are $\lg N$ passes, using about $N$ compares, for a total cost of about $N \lg N$.

In [3]:
def mergeSortBU(arr):
    N = len(arr)
    for sz in [2**i for i in range(N) if 2**i < N]:
        for lo in range(0, N - sz, sz + sz):
            mid = sz - 1
            hi = min(lo + sz + sz - 1, N - 1)
            arr[lo:hi + 1] = merge(arr[lo:hi + 1], mid)
    return arr

d = ['A', 'G', 'L', 'O', 'R', 'H', 'I', 'M', 'S', 'T']
print("Bottom-up mergesort: {}".format(mergeSortBU(d)))

Bottom-up mergesort: ['A', 'G', 'H', 'I', 'L', 'M', 'O', 'R', 'S', 'T']


## Sorting Complexity

**Computational complexity** is a framework to study efficiency of algorithms for solving a particular problem $X$. To evaluate the complexity of sorting (or any task $X$), you need:

- Model of computation: allowable operations
- Cost model: operation count(s)
- Upper bound: cost guarantee provided by some algorithm for $X$
- Lower bound: proven limit on cost guarantee of all algorithms for $X$
- Optimal algorithm: algorithm with best possible cost guarantee for $X$ (ideally, find one where the upper and lower bound are the same)

Sorting uses a decision tree as the model of computation, the cost is the number of compares, the upper bound is ~$N \lg N$ from mergesort, and start with an unknown lower bound or optimal algorithm.

With a decision tree, make a comparison (first element to second), then branch whether the first element is less than the second element or not. The **height** of the tree is the **worst-case number of compares**. So for compare-based sorting, any algorithm must use at least $\lg (N!) \text{ ~ } N \lg N$ (by Stirling's formula) compares in the worst case.

Proof:

- Assume the array consists of $N$ distinct values $a_1$ through $a_N$
- The worst case is dictated by height $h$ of the decision tree
- Binary tree of height $h$ has at most $2^h$ leaves
- $N!$ different orderings $\implies$ at least $N!$ leaves

$$
2^h \geq \text{ # leaves } \geq N! \\
\implies h \geq \lg (N!) \text{ ~ } N \lg N
$$

Therefore, the lower bound is ~$N \lg N$, which is the same as the upper bound, and the optimal algorithm is mergesort. Note, mergesort is optimal with respect to the number of compares, but it's NOT optimal with respect to space usage. What you can takeaway is you shouldn't try to find fewer number of compares than mergesort, as it hits the lower bound. But you can try to find one with better space usage, while maintaining the number of compares.

Note also that the complexity results are only for number of compares - that lower bound may not hold if the algorithm has information about the initial order of the input, the distribution of key values, or the representation of the keys. For example, you may not need $N \lg N$ compares for a partially-ordered array or when one has duplicate keys.

## Stability

**Stability** is the property where a sort preserves the relative order of the items with duplicate keys. This is relevant when you first sort by one field, then by a subfield (like last name then first name). Not all sort implementations are stable.

Stable:

- Insertion sort
- Mergesort

Not stable:

- Selection sort
- Shellsort

You can tell by carefully checking the code for "less than" or "less than or equal to".

**Insertion sort is stable** because you never move an equal item past each other. **Selection sort and Shellsort are not stable** because long-distance exchanges might move an item past some equal item. **Mergesort is stable**, as long as the merge operation is, which depends on how it's implemented (takes from the left subarray if the keys are equal).